In [1]:
import pickle, pandas as pd, numpy as np, os

In [ ]:
with open('/ssd2/ming/2024COVID/train_dic1_05_challenge.pickle', 'rb') as f:
    train_dic = pickle.load(f)
with open('/ssd2/ming/2024COVID/valid_dic1_05_challenge.pickle', 'rb') as f:
    valid_dic = pickle.load(f)
print(len(train_dic),len(valid_dic))

In [5]:
replace_list = ["extra_challenge1_non_cov_train_new_icassp","extra_challenge1_non_cov_val_new_icassp","extra_challenge1_cov_val_new_icassp"]
copy_train_dic = {}
for keys_, value in train_dic.items():
    if "extra_challenge1_non_cov_train_new_icassp" in keys_:
        old_path = "/ssd2/ming/2024COVID/train/negative/"
        new_path = "/ssd2/ming/2024COVID/train_pure_crop_challenge/negative/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_train_dic[new_key] = value
train_dic = copy_train_dic
copy_train_dic = {}
for keys_, value in train_dic.items():
    if "extra_challenge1_non_cov_train_new_icassp" in keys_:
        old_path = "/ssd2/ming/2024COVID/train/positive/"
        new_path = "/ssd2/ming/2024COVID/train_pure_crop_challenge/positive/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_train_dic[new_key] = value
train_dic = copy_train_dic
copy_valid_dic = {}
for keys_, value in valid_dic.items():
    if "extra_challenge1_non_cov_val_new_icassp" in keys_:
        old_path = "/ssd2/ming/2024COVID/valid/negative/"
        new_path = "/ssd2/ming/2024COVID/valid_pure_crop_challenge/negative/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_valid_dic[new_key] = value
valid_dic = copy_valid_dic
copy_valid_dic = {}
for keys_, value in valid_dic.items():
    if "extra_challenge1_cov_val_new_icassp" in keys_:
        old_path = "/ssd2/ming/2024COVID/valid/positive/"
        new_path = "/ssd2/ming/2024COVID/valid_pure_crop_challenge/positive/"
        new_key = keys_.replace(old_path, new_path)
    else:
        new_key = keys_
    copy_valid_dic[new_key] = value
valid_dic = copy_valid_dic

In [12]:
len(train_dic)

1358

In [ ]:
train_df = pd.DataFrame(None, columns=['path', 'filename', 'label'])
valid_df = pd.DataFrame(None, columns=['path', 'filename', 'label'])

print("="*10, "loading data DataFrame", "="*10)
for path in list(train_dic.keys()):
    image_list = os.listdir(path)
    image_list.sort(key=lambda x: int(os.path.splitext(x)[0]))
    temp_df = pd.DataFrame([path]*len(image_list), columns=['path'])
    temp_df['filename'] = image_list
    if 'negative' in temp_df.path[0]:
        temp_df['label'] = [0]*len(image_list)
    else:
        temp_df['label'] = [1]*len(image_list)
    train_df = pd.concat([train_df, temp_df])

for path in list(valid_dic.keys()):
    image_list = os.listdir(path)
    image_list.sort(key=lambda x: int(os.path.splitext(x)[0]))
    temp_df = pd.DataFrame([path]*len(image_list), columns=['path'])
    temp_df['filename'] = image_list
    if 'negative' in temp_df.path[0]:
        temp_df['label'] = [0]*len(image_list)
    else:
        temp_df['label'] = [1]*len(image_list)
    valid_df = pd.concat([valid_df, temp_df]) 

In [18]:
import copy
cp_train_dic = copy.deepcopy(train_dic)
cp_valid_dic = copy.deepcopy(valid_dic)
print(len(train_dic), len(valid_dic))
print(train_df.shape, valid_df.shape)

1358 326
(384916, 3) (89671, 3)


In [21]:
drop_count_t = 0
filter_num = 5
for path_ in train_dic:
    i = len(os.listdir(path_))
    if i <filter_num:
        print(f"張數{i}", path_)
        drop_count_t = drop_count_t + i
        del cp_train_dic[path_]
        train_df = train_df[train_df.path!=path_]
drop_count_v = 0
for path_ in valid_dic:
    i = len(os.listdir(path_))
    if i <filter_num:
        print(f"張數{i}", path_)
        drop_count_v = drop_count_v + i
        del cp_valid_dic[path_]
        valid_df = valid_df[valid_df.path!=path_]

print(len(cp_train_dic), len(cp_valid_dic))
print(train_df.shape, valid_df.shape)
print(drop_count_t, drop_count_v)

張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/negative/ct_scan_292
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/negative/ct_scan_354
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/negative/ct_scan_449
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/negative/ct_scan_538
張數4 /ssd2/ming/2024COVID/train_pure_crop_challenge/positive/ct_scan_107
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/positive/ct_scan_306
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/positive/ct_scan_31
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/positive/ct_scan_47
張數1 /ssd2/ming/2024COVID/train_pure_crop_challenge/positive/ct_scan_64
張數1 /ssd2/ming/2024COVID/valid_pure_crop_challenge/negative/ct_scan_101
張數1 /ssd2/ming/2024COVID/valid_pure_crop_challenge/negative/ct_scan_130
張數1 /ssd2/ming/2024COVID/valid_pure_crop_challenge/negative/ct_scan_15
張數1 /ssd2/ming/2024COVID/valid_pure_crop_challenge/positive/ct_scan_101
張數1 /ssd2/ming/2024COVID/valid_pure_crop_challenge/positive/ct_scan_

In [27]:
train_df = train_df[['path', 'label']]
valid_df = valid_df[['path', 'label']]
train_df = train_df.drop_duplicates(subset='path')
valid_df = valid_df.drop_duplicates(subset='path')
print(train_df.shape, valid_df.shape)

(1349, 2) (318, 2)


In [28]:
with open('/ssd2/ming/2024COVID/filter_slice_train_dic1_05_challenge.pickle', 'wb') as handle:
    pickle.dump(cp_train_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/ssd2/ming/2024COVID/filter_slice_valid_dic1_05_challenge.pickle', 'wb') as handle:
    pickle.dump(cp_valid_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
train_df.to_csv("/ssd2/ming/2024COVID/filter_slice_train_df_challenge.csv", index=False, encoding='utf-8-sig')
valid_df.to_csv("/ssd2/ming/2024COVID/filter_slice_valid_df_challenge.csv", index=False, encoding='utf-8-sig')